### Basic aperture photometry using weirdly-shaped apertures, using ds9 regions files, and pyregion to filter the images by those regions

This adapts my pyregion tutorial, to do simple photometry of HST for SGAS J0033.

In [45]:
import numpy as np
from astropy.io import fits
from astropy.utils.data import download_file
from astropy.io.fits import getdata
import pyds9
import pyregion

In [46]:
# Set up locations
imdir = "/Users/jrrigby1/Dropbox/S0033/HST/"
imfile1 = "final_images/SGAS0033_F410M_full_0.03g0.5_drc_sci.fits"
imfile2 = "final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits"
d = pyds9.DS9('foo1')  # start ds9.  'd' is the way to call ds9
#d = pyds9.DS9('809ab48d:63115')

In [57]:
def adjust_ds9():
    d.set("colorbar no")   # example of manipulating the ds9 window
    d.set("scale zscale")  # example of manipulating the ds9 window
    d.set("zoom to 1.5 1.5")
    d.set("pan to 2460 1920")
    return(0)

def pyregions_photometry(object_region, sky_region, im) :  # This was workaround for old pyreg
    myfilter   = object_region.get_filter()
    bkgfilter  = sky_region.get_filter()
    total_flux = np.nansum(myfilter.mask(im) * im)  #nansum treats nans as zeros for summation
    npix_reg   = np.count_nonzero(myfilter.mask(im))
    npix_bkg   = np.count_nonzero(bkgfilter.mask(im))
    bkg_med    = np.nanmean(bkgfilter.mask(im) * im)
    bkg_flux   = bkg_med * npix_reg / npix_bkg
    net_flux   = total_flux - bkg_flux
    print "Npix in regions", npix_reg, npix_bkg
    return(total_flux, bkg_flux, net_flux)

def pyregions_photometry2(regfile, bkgregfile, imagefile):
    f = fits.open(imagefile)
    dum = f[0].data  # screwing around to get hdu as pyregions wants it
    im = dum.astype(np.float64)
    d.set_np2arr(im) # sending ndarray im directly to ds9
    reg = pyregion.open(regfile).as_imagecoord(f[0].header)
    bkg = pyregion.open(bkgregfile).as_imagecoord(f[0].header)
    d.set('regions load', regfile)
    d.set('regions load', bkgregfile)
    npix_reg   = np.count_nonzero(reg.get_mask(f[0]) * im)
    npix_bkg   = np.count_nonzero(bkg.get_mask(f[0]) * im)
    total_flux = np.nansum(reg.get_mask(f[0]) * im)
    bkg_mean   = np.nanmean(bkg.get_mask(f[0]) * im)
    bkg_flux   = bkg_mean * npix_reg / npix_bkg
    net_flux = total_flux - bkg_flux
#    return(total_flux, bkg_flux, net_flux)
    return(net_flux)

In [61]:
filters = ('F555W', 'F410M')
AB_zpt  = (25.824, 23.614)
images  = (imfile2, imfile1)
for jj, image in enumerate(images):
    for ii in range(1,7+1,1) :
        net_flux = pyregions_photometry2(imdir + 'Regions/mike_phot555_'+str(ii)+'.reg', imdir + 'Regions/mike_phot555_bkg.reg', imdir + image)
        mAB = -2.5*np.log10(net_flux) + AB_zpt[jj]
        print image, filters[jj], ii, net_flux, mAB

final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits F555W 1 4.1860940905 24.2694775357
final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits F555W 2 7.73839853997 23.6023722679
final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits F555W 3 3.65794037419 24.4159084449
final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits F555W 4 34.0169399509 21.9947618916
final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits F555W 5 13.5358312237 22.9952876736
final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits F555W 6 6.82633233118 23.7385314312
final_images/SGAS0033_F555W_0.03g0.5_drc_sci.fits F555W 7 35.6442146447 21.9440273743
final_images/SGAS0033_F410M_full_0.03g0.5_drc_sci.fits F410M 1 0.532778037844 24.2976342148
final_images/SGAS0033_F410M_full_0.03g0.5_drc_sci.fits F410M 2 0.20537411793 25.3326357221
final_images/SGAS0033_F410M_full_0.03g0.5_drc_sci.fits F410M 3 0.384537234989 24.651654002
final_images/SGAS0033_F410M_full_0.03g0.5_drc_sci.fits F410M 4 5.38035884227 21.7869718954
final_images/SGAS0033_F410M_full_0

In [68]:
''' As a side result, can now figure out how to cont-sub the F410M image
F410M has bandpass of 70A.    Observed EW=203A.  So, f(Lya)=2.9*f(cont)
f(Lya) = (F410M) - f(cont)
f(F410M) = f(Lya) + f(cont)
         = f(Lya) + 1/2.9*f(Lya)
         = 1.345 f(Lya)
f(Lya) = 0.744 f(F410M) 
'''
cts_F555W = 35.6442146447  # Copy and pasted from above
cts_F410M = 5.67078046693

corr_factor = (cts_F410M / cts_F555W) / 1.345
print "Try mutiplying the F555W image by", corr_factor, ", and use that to continuum subtract F410M"
# Looks pretty good.  Done this task.

Try mutiplying the F555W image by 0.118285497317 , and use that to continuum subtract F410M


Let's translate above into plain English:  F410M was the medium band filter, and F555W the continuum filter. Given the high equivalent width of Lyman alpha in the MagE spectrum, EW_obs=203A, and the F410M bandpass of 70A, we calculate that Lyman alpha contributes 74% of the flux in F410M, with the remainder coming from continuum.  We then scale the F555W image to match that continuum level, using annular aperture photometry of SGAS0033 in the F410M and F555W HST images, covering the same region as the MagE aperture. (sent this to Travis 2/19/2019 to address referee comment.)